In [ ]:
import pandas as pd
df_res = pd.read_csv("data.csv", header=None)
import pickle
images = pickle.loads(df_res.loc[0,1])
index = pickle.loads(df_res.loc[0,2])
actions = pickle.loads(df_res.loc[0,3])
actions_new = pickle.loads(df_res.loc[0,4])
sub_action = actions_new[actions_new[:,0] == 0, :]
sub_image = images[:,:,actions_new[:,0] == 0]

In [1]:
import pandas as pd
import pickle

sub_image = pickle.load( open( "images1.txt", "rb" ) )
sub_action = pickle.load( open( "actions1.txt", "rb" ) )

In [2]:
def f(x): return x >30 and x < 60 and x%4 != 0 #x % 3 == 0 or x % 5 == 0
ind = filter(f, range(80))
def g(x): return x%2 != 0 #x % 3 == 0 or x % 5 == 0
ind_w = filter(g, range(80))

In [3]:
def f(x): return x%7== 0 #x % 3 == 0 or x % 5 == 0
ind7 = filter(f, range(sub_action.shape[0]-1))

In [ ]:
ind7[1]

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(images[ind,:,239], cmap='gray')

In [ ]:
cropped_tmp = images[ind,:,:]
cropped = cropped_tmp[:,ind_w,:]
# plt.imshow(cropped[:,ind_w], cmap='gray')
cropped_tmp.shape

In [ ]:
plt.imshow(cropped[:,:,1000], cmap='gray')

In [ ]:
plt.imshow(images[:,:,1000], cmap='gray')

In [5]:
y = sub_action

In [6]:
import numpy as np
y_target = np.zeros(y.shape[0])
for i in range(y_target.shape[0]):
    y_target[i]= sub_action[i].nonzero()[0][0]

In [7]:
sub_image.shape

(80, 80, 6156)

In [8]:
X = sub_image
cropped_tmp = X[ind,:,:]
cropped = cropped_tmp[:,ind_w,:]

x_turned =cropped.transpose(2,0,1).reshape(y_target.shape[0],-1)
# x_turned = x_turned[ind7]
# x_turned = np.rot90(np.flip(np.swapaxes(np.swapaxes(cropped, 2,0), 0,1),1),k=3)

In [ ]:
cropped.shape

In [ ]:
x_turned[ind7].shape

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

kfold = StratifiedKFold(n_splits= 5, shuffle= True)

clf = LogisticRegressionCV()

cv = kfold.split(x_turned[ind7], y_target[ind7])

scores = cross_val_score(clf, x_turned[ind7], y_target[ind7], scoring='accuracy', cv=cv)

print ('StratifiedKFold score: {0:.3f} ± {1:.3f}'.format(scores.mean(), scores.std()) )

In [9]:
print(len(sub_action))
dim = len(sub_action)
arr = np.arange(dim)
np.random.shuffle(arr)
arr

6156


array([1833, 3274, 1655, ..., 1050,   36, 1638])

In [10]:
dim = len(sub_action)
arr = np.arange(dim)
arr = arr[:len(sub_action)/7]
arr.shape

(879,)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x_turned[arr], y_target[arr])

In [17]:
LR = LogisticRegression(multi_class='ovr')

In [18]:
LR.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
float(np.sum(LR.predict(x_test) != y_test))/y_test.shape[0]

0.00909090909090909

In [20]:
y_test.shape[0]

220

In [21]:
%env THEANO_FLAGS=device=cpu,floatX=float32

env: THEANO_FLAGS=device=cpu,floatX=float32


In [22]:
import gym
import ppaquette_gym_doom
import matplotlib.pyplot as plt
from gym.wrappers import SkipWrapper
from ppaquette_gym_doom.wrappers.action_space import ToDiscrete


%matplotlib inline
GAME_NAME = 'ppaquette/DoomBasic-v0'

env = ToDiscrete('minimal')(gym.make(GAME_NAME))



INFO:gym.envs.registration:Making new env: ppaquette/DoomBasic-v0
[2017-03-21 10:02:49,730] Making new env: ppaquette/DoomBasic-v0


In [23]:
def_state = env.reset()

def_state_80_rg = np.array(toimage(def_state).resize((80,80), Image.ANTIALIAS).convert('L'))

ds80_1 = def_state_80_rg[ind,:]
ds80_2 = ds80_1[:,ind_w]
LR.predict(ds80_2.reshape(1,-1))
new_state = def_state

NameError: name 'toimage' is not defined

In [ ]:
processed_state = np.array(toimage(new_state).resize((80,80), Image.ANTIALIAS).convert('L'))
processed_state = processed_state[ind,:]
processed_state = processed_state[:,ind_w]
processed_state = processed_state.reshape(1,-1)
action = LR.predict(processed_state)[0]
print action
new_state = env.step(int(action))[0]

In [ ]:
plt.imshow(new_state)

In [ ]:
def_state = env.reset()

i = 10
if i % 4 == 2:
    print("hello")

In [24]:
episodes = 200
#     index = np.empty([1, 1])
import time
from scipy.misc import toimage
from PIL import Image
rew = []
ti = []

for j in range(1):
    print("")
    rewards = []
    time_val = []
    for i in range(episodes):

        start_time = time.time()

#         if i+1 % 25 == 2:
#             print("episode #", i)

        def_state = env.reset()
        is_done = False

        def_state_80_rg = np.array(toimage(def_state).resize((80,80), Image.ANTIALIAS).convert('L'))

        ds80_1 = def_state_80_rg[ind,:]
        ds80_2 = ds80_1[:,ind_w]
        action = LR.predict(ds80_2.reshape(1,-1))
        tot_rew = 0

        st_num = 0
    #     game.new_episode()
        while not is_done:

            st_num += 1
            new_env = env.step(int(action))
            new_state = new_env[0]
            is_done = new_env[2]

            r = new_env[1]

            processed_state = np.array(toimage(new_state).resize((80,80), Image.ANTIALIAS).convert('L'))
            processed_state = processed_state[ind,:]
            processed_state = processed_state[:,ind_w]
            processed_state = processed_state.reshape(1,-1)
            action = LR.predict(processed_state)[0]

            tot_rew += r



    #         game.advance_action()
    #         a = game.get_last_action()
    #         r = game.get_last_reward()

    #         if (s.number % frame_sk == 0):
    #             im_ar = np.array(toimage(img).resize((80,80), Image.ANTIALIAS).convert('L'))
    #             images = np.dstack((images, im_ar))
    #             actions = np.append(actions, np.atleast_2d(a), axis=0)
    #             rew_step.append(r)




    #         imgs.append(img)
    #         actions.append(a)

#             print("state #", st_num)
#             #print("game variables: ", misc)
#             print("action:", action)
#             print("reward:",r)
#             print("=====================")

        ep_time = time.time() - start_time
        print("episode finished!,{}".format(i))
        print("total reward: {}, time:{}".format(tot_rew,ep_time))

        rewards.append(tot_rew)
        time_val.append(ep_time)
    #     print("************************")
    print np.mean(rewards)
    print np.std(rewards)
    print np.mean(ep_time)
    print np.std(ep_time)
    ti.append([np.mean(ep_time), np.std(ep_time)])
    rew.append([np.mean(rewards), np.std(rewards)])


episode finished!,0
total reward: 70.0, time:0.300343990326
episode finished!,1
total reward: 78.0, time:0.190007925034
episode finished!,2
total reward: -470.0, time:2.56851100922
episode finished!,3
total reward: 76.0, time:0.249701023102
episode finished!,4
total reward: -465.0, time:2.36446094513
episode finished!,5
total reward: 68.0, time:0.183190107346
episode finished!,6
total reward: 75.0, time:0.145995140076
episode finished!,7
total reward: 71.0, time:0.177366018295
episode finished!,8
total reward: 93.0, time:0.0526051521301
episode finished!,9
total reward: 65.0, time:0.199979066849
episode finished!,10
total reward: -470.0, time:1.89336705208
episode finished!,11
total reward: 84.0, time:0.0987911224365
episode finished!,12
total reward: 94.0, time:0.0457670688629
episode finished!,13
total reward: 85.0, time:0.0906870365143
episode finished!,14
total reward: 90.0, time:0.0675699710846
episode finished!,15
total reward: 76.0, time:0.141279935837
episode finished!,16
tota

In [ ]:
print np.mean(rewards)
print np.std(rewards)

np.swapaxes(np.asarray(rew),1,0)

In [ ]:
np.mean(np.asarray(rew), axis = 0)

In [ ]:
st = env.step(2)[0]

In [ ]:
plt.imshow(st)

In [ ]:
rewards

In [ ]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovr',kernel = 'linear', C = 0.000001)
clf.fit(x_train, y_train) 

In [ ]:
np.sum(clf.predict(x_test) != y_test)

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

from sklearn.model_selection import GridSearchCV
svr = svm.SVC(decision_function_shape='ovr')
clf = GridSearchCV(svr, param_grid)

In [ ]:
clf.fit(x_train, y_train) 

In [ ]:
np.sum(clf.predict(x_test) != y_test)